다음의 커널을 필사했습니다.  
https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution/data  
https://www.youtube.com/watch?v=aoo1xrKQXFc&list=PLC_wC_PMBL5PexxXb6UC6pDXODE4xFWzL

In [123]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [124]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

In [125]:
warnings.simplefilter(action='ignore', category = FutureWarning)

if __name__ == "__main__":  
    pd.set_option('display.max_rows', 60)  
    pd.set_option('display.max_columns', 100)  
    with timer("Pipeline total time"):  
        main(debug= False)  
        
위의 코드로 실행이 되는데 이 노트북에서는 스크립트 형으로 진행하지 않기 때문에 처음부터 설정을 해줍니다.

In [126]:
pd.set_option('display.max_rows', 60)

In [127]:
pd.set_option('display.max_columns', 100)

In [128]:
# main 함수에서 debug = False로 돼있지만 지금은 True이기 때문에 인위적으로 True로 해줍니다.

debug = True
num_rows = 30000 if debug else None

## df = get_train_test(DATA_DIRECTORY, num_rows = num_rows) # 함수 뜯어보기

In [129]:
# GENERAL CONFIGURATIONS
NUM_THRADS = 4
DATA_DIRECTORY = "../input/"
SUBMISSION_SUFIX = "_model2_04"

In [130]:
path = DATA_DIRECTORY
num_rows = num_rows

In [131]:
train_file_path = '../input/home-credit-default-risk/application_train.csv'
test_file_path = '../input/home-credit-default-risk/application_test.csv'

In [132]:
train = pd.read_csv(train_file_path, nrows = num_rows)
test = pd.read_csv(test_file_path, nrows = num_rows)

In [133]:
df = train.append(test)

In [134]:
print(train.shape)

In [135]:
train.head()

In [136]:
train['TARGET'].value_counts().plot.bar()

In [137]:
del train, test

# 메모리 주워 담기
gc.collect

가장 먼저 target값이 imbalance한지 봐야합니다.

# Preprocessing

원래는 EDA를 먼저 들어가야 하지만 여기서는 해당 파트에 대한 설명이 없기 때문에 Preprocessing에 바로 들어갑니다.

In [138]:
# 전체 데이터 프레임을 봤을 때 XNA에는 4명의 사람 밖에 없기 때문에 제거합니다. (아웃라이어로 판단)
df = df[df['CODE_GENDER'] != 'XNA']

In [139]:
# 2천만 이하는 100개 정도 밖에 없기 때문에 이 또한 제거합니다.
df = df[df['AMT_INCOME_TOTAL'] < 20000000]

365243으로 기록된 사람들이 많은데, 이는 의미 없는 숫자로 보입니다. 하지만 이를 제거하면 문제가 생길 수 있습니다. 따라서 일단 nan값을 적용하겠습니다.

In [140]:
df

In [141]:
df[df['DAYS_EMPLOYED'] == 365243].value_counts()

In [142]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [143]:
# 핸드폰 바꾼지 0일이라는 것은 null 값을 무언가로 채웠음을 의미하는 것 같습니다. 따라서 nan으로 대체합니다.
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [146]:
# 이들은 binary data입니다. "에, 아니오" 등으로 이루어진 문서
docs = [f for f in df.columns if 'FLAG_DOC' in f]

In [149]:
docs

In [154]:
df[docs].kurtosis(axis=1).hist()

In [150]:
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1) # 설문조사에서 "예"를 몇 번 했는지 체크
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1) # 왜곡된 정도를 컬럼으로 추가

In [148]:
df[docs].sum(axis=1)